In [173]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [174]:
data = pd.read_csv('data/cheap_train_sample.csv')
data.shape

(6513, 14)

In [175]:
test_data = pd.read_csv('data/test_data.csv') # from which to make predictions for the competition
test_data.shape

(16281, 13)

# 1) Pre-process data

Concatenate train and test data so that that two are pre-processed the same way. 

In [176]:
concat_train_test = pd.concat([data,test_data])

## 1.1) Binarize Target Feature

In [177]:
# the .replace method below does not change the vaues from the test set (NaNs)
concat_train_test['wage'] = concat_train_test['wage'].replace({' <=50K':0, ' >50K':1})
# make sure the new values are 0,1, and NaN (for test set)
concat_train_test['wage'].unique()

array([ 0.,  1., nan])

## 1.2) Resolve 'never_worked' with 'hours_per_week'

For all rows with `work-class` in  [ 'never worked' ,'without pay'],  set `hours_per_week` to 0.


In [178]:
# create a filter disjunction
filter1 = (concat_train_test['workclass']==' Without-pay')
filter2 = (concat_train_test['workclass']==' Never-worked'  )
filter_disjunction = filter1 | filter2

#write zeros
concat_train_test.loc[filter_disjunction,['hours-per-week']] = 0

# make sure zeros were written
concat_train_test.loc[filter_disjunction,['hours-per-week']]

,hours-per-week
2019,0
4124,0
4167,0
5944,0
2957,0
3177,0
6466,0
8785,0
8903,0
10647,0


## 1.3) Dummy Features

In [179]:
dumb_feats = ['workclass','marital-status','occupation','sex']
concat_train_test = pd.get_dummies(concat_train_test, 
               columns = dumb_feats, 
               drop_first=True
              )

In [180]:
#view the new feature columns
concat_train_test.head().T


,0,1,2,3,4
age,56,28,33,26,40
fnlwgt,346033,96226,251120,178140,56795
education,9th,HS-grad,Bachelors,Bachelors,Masters
education-num,5,9,13,13,14
relationship,Not-in-family,Husband,Husband,Husband,Not-in-family
capital-gain,0,0,7688,0,14084
capital-loss,0,0,0,0,0
hours-per-week,40,45,50,45,55
native-country,United-States,United-States,United-States,United-States,United-States
wage,0.0,0.0,1.0,1.0,1.0


## 1.4) Engineer quadratic feature

Since greater education likely linearly correlates to hourly py rate, income likley correlates with education times hours worked per week, a quadratic feature.

In [181]:
concat_train_test['ed_num_x_hours'] = concat_train_test['education-num'] * concat_train_test['hours-per-week']

## 1.5) Drop Features

In [182]:
drop_feats = ['fnlwgt', 'education',
              'relationship', #'capital-gain','capital-loss',
              'native-country',
             ]

concat_train_test.drop(columns=drop_feats, inplace = True)

# 2) Re-Split train and test data

In [183]:
data = concat_train_test.iloc[:len(data),:] #dummied, dropped
test_data = concat_train_test.iloc[len(data):] # dummied, dropped

# 3) Split X & Y  (predictor & target)

In [184]:
X = data.drop(columns = ['wage'])
y = data['wage']

For the test data, there is no target data. 

In [185]:
X_test = test_data.drop(columns = ['wage'])
# y_test will be predicted below and assesed by judges.

## 3.1) Train-Validation split (with `train_test_split()`)

In [186]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(X,y,random_state=8,shuffle=True ,stratify=y)

In [207]:
len(y_train),y_train.value_counts()

(4884,
 0.0    3708
 1.0    1176
 Name: wage, dtype: int64)

# 4) Consider Balancing Train Data via Upsampling

We will upsample the >50k group in the training data to deal with the fact that 76% of the data points are for the <=50k group. Note that we do not balance the data of the validation set, correcting a mistake made durring the competition. 

In [187]:
# examine class balance
y_train.value_counts(normalize=True) 

0.0    0.759214
1.0    0.240786
Name: wage, dtype: float64

In [188]:
train_data = pd.concat([X_train,y_train], axis=1)

In [189]:
from sklearn.utils import resample

# rows with wage = 0 (76%)
under_50k = train_data[ train_data['wage'] == 0]

#rows with wage =1 (24%)
over_50k = train_data[train_data['wage'] == 1]

#augment B to have the same number of rows as A
over_50k_aug = resample(over_50k, 
                replace = True,
                n_samples = len(under_50k),
                 random_state = 14
                )

train_data_balanced = pd.concat([under_50k,over_50k_aug] )
train_data_balanced['wage'].value_counts(normalize=True)

0.0    0.5
1.0    0.5
Name: wage, dtype: float64

In [190]:
# comment out this cell to use unbalanced data
# X_train = train_data_balanced.drop(columns = ['wage'])
# y_train = train_data_balanced['wage']

**Post moretem:** We opt to not use the balanced train data below; the input to the `.fit()` methods below 
- are `X_train`, `y_train`, 
- are not `X_train_bal`, `y_train_bal`


# 5) Model Pipes to Stack

**Plan**: Grid search for each model type, then stack.

Note that we scale data in pipelines instead of before feeding scaled data to pipelines. 

In [191]:
import time 
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

### Logistic Regression

In [192]:
%%time
from sklearn.linear_model import LogisticRegression

pipe_logreg = Pipeline([
    ('ss',StandardScaler()),
    ('logreg',LogisticRegression(penalty='l2' , max_iter=10**5) )
     ])

grid_logreg = {
    'logreg__penalty': ['l2','none'],
}
gs_logreg = GridSearchCV(pipe_logreg, 
                         param_grid= grid_logreg,
                        )

gs_logreg.fit(X_train,y_train) # we use unbalanced data to train

print(f'Train score: {gs_logreg.score(X_train,y_train)}')
print(f'Test score: {gs_logreg.score(X_val,y_val)}')
gs_logreg.best_score_, gs_logreg.best_params_ 

Train score: 0.8476658476658476
Test score: 0.8502148557397177
CPU times: user 1.39 s, sys: 455 ms, total: 1.85 s
Wall time: 373 ms


(0.8472555246069433, {'logreg__penalty': 'l2'})

### kNN

In [193]:
%%time
from sklearn.neighbors import KNeighborsClassifier

pipe_knn = Pipeline([
    ('ss',StandardScaler()),
    ('knn' , KNeighborsClassifier(n_neighbors=10,weights='uniform') )
])

grid_knn = {'knn__n_neighbors':[8,9,10,11,12],
            'knn__weights': ['uniform','distance']
           }

gs_knn =GridSearchCV(pipe_knn,
                    param_grid=grid_knn)

gs_knn.fit(X_train, y_train)
print(f'Train score: {gs_knn.score(X_train,y_train)}')
print(f'Test score: {gs_knn.score(X_val,y_val)}')

gs_knn.best_score_ , gs_knn.best_params_

Train score: 0.8468468468468469
Test score: 0.8354818907305095
CPU times: user 18.4 s, sys: 5.96 s, total: 24.4 s
Wall time: 4.34 s


(0.8230980586271122, {'knn__n_neighbors': 12, 'knn__weights': 'uniform'})

### Decision Tree

In [194]:
%%time

from sklearn.tree import DecisionTreeClassifier


pipe_dtc = Pipeline([
    ('ss',StandardScaler()),
    ('dtc',DecisionTreeClassifier(max_depth=5,min_samples_split=5) )
     ])

grid_dtc = {
    'dtc__max_depth': [None,1,5,10,15], 
    'dtc__min_samples_split': [5,6,7,8],
}
gs_dtc = GridSearchCV(pipe_dtc, 
                         param_grid= grid_dtc,
                        )
gs_dtc.fit(X_train,y_train)
print(f'Train score: {gs_dtc.score(X_train,y_train)}')
print(f'Test score: {gs_dtc.score(X_val,y_val)}')

gs_dtc.best_score_, gs_dtc.best_params_


Train score: 0.8445945945945946
Test score: 0.8397790055248618
CPU times: user 1.17 s, sys: 91.1 ms, total: 1.26 s
Wall time: 1.03 s


(0.8343561756464251, {'dtc__max_depth': 5, 'dtc__min_samples_split': 5})

### Bagging Trees

In [195]:
%%time

from sklearn.ensemble import BaggingClassifier

pipe_btc = Pipeline([
    ('ss',StandardScaler()),
    ('btc',BaggingClassifier(n_estimators=150) )
     ])

grid_btc = {
    'btc__n_estimators': [150],
#     'btc__max_depth': [None,1,5,10,15], 
#     'btc__min_samples_split': [5,6,7,8],
}
gs_btc = GridSearchCV(pipe_btc, 
                         param_grid= grid_btc,
                        )
gs_btc.fit(X_train,y_train)
print(f'Train score: {gs_btc.score(X_train,y_train)}')
print(f'Test score: {gs_btc.score(X_val,y_val)}')

gs_btc.best_score_, gs_btc.best_params_

Train score: 0.9885339885339886
Test score: 0.8520564763658687
CPU times: user 6.56 s, sys: 38.3 ms, total: 6.6 s
Wall time: 6.63 s


(0.8259635552125106, {'btc__n_estimators': 150})

### Random Forest

In [196]:
%%time
from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier()

pipe_rfc = Pipeline([
    ('ss',StandardScaler()),
    ('rfc',RandomForestClassifier(max_depth=10, n_estimators=200) )
     ])

grid_rfc = {
    'rfc__n_estimators': [100,150,200],
    'rfc__max_depth': [6,8,10,12,14,16],
}
gs_rfc = GridSearchCV(pipe_rfc, 
                         param_grid= grid_rfc,
                        )
gs_rfc.fit(X_train,y_train)
print(f'Train score: {gs_rfc.score(X_train,y_train)}')
print(f'Test score: {gs_rfc.score(X_val,y_val)}')

gs_rfc.best_score_, gs_rfc.best_params_ #cvs.832 {'rfc__max_depth': 10, 'rfc__n_estimators': 200})


Train score: 0.9021294021294022
Test score: 0.8643339472068754
CPU times: user 24.6 s, sys: 152 ms, total: 24.7 s
Wall time: 24.8 s


(0.8519648640032216, {'rfc__max_depth': 12, 'rfc__n_estimators': 150})

### AdaBoost Logreg

In [197]:
%%time
from sklearn.ensemble import AdaBoostClassifier
ada_logreg = AdaBoostClassifier(base_estimator=LogisticRegression())

pipe_ada_logreg = Pipeline([
    ('ss',StandardScaler()),
    ('ada_logreg', AdaBoostClassifier(base_estimator=LogisticRegression(),
                                      learning_rate=1.1,n_estimators = 10),
    )
     ])

grid_ada_logreg = {
'ada_logreg__learning_rate': [0.9,1.0,1.1],
 'ada_logreg__n_estimators': [10,50,100],
}
gs_ada_logreg = GridSearchCV(pipe_ada_logreg, 
                         param_grid= grid_ada_logreg,
                        )
gs_ada_logreg.fit(X_train,y_train)
print(f'Train score: {gs_ada_logreg.score(X_train,y_train)}')
print(f'Test score: {gs_ada_logreg.score(X_val,y_val)}')

gs_ada_logreg.best_score_, gs_ada_logreg.best_params_ # 0.8296472557343492

Train score: 0.8452088452088452
Test score: 0.8453038674033149
CPU times: user 54 s, sys: 14.7 s, total: 1min 8s
Wall time: 10 s


(0.843775273923184,
 {'ada_logreg__learning_rate': 0.9, 'ada_logreg__n_estimators': 100})

### AdaBoost Decision Tree

In [198]:
%%time
from sklearn.ensemble import AdaBoostClassifier
# ada_tree = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
pipe_ada_tree = Pipeline([
    ('ss',StandardScaler()),
    ('ada_tree', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),learning_rate =0.9,n_estimators=100) )
     ])

grid_ada_tree = {
'ada_tree__learning_rate': [0.9,1.0,1.1],
 'ada_tree__n_estimators': [10,50,100],
}
gs_ada_tree = GridSearchCV(pipe_ada_tree, 
                         param_grid= grid_ada_tree,
                        )
gs_ada_tree.fit(X_train,y_train)
print(f'Train score: {gs_ada_tree.score(X_train,y_train)}')
print(f'Test score: {gs_ada_tree.score(X_val,y_val)}')

gs_ada_tree.best_score_, gs_ada_tree.best_params_


Train score: 0.9885339885339886
Test score: 0.8238182934315531
CPU times: user 21 s, sys: 389 ms, total: 21.4 s
Wall time: 20.3 s


(0.8165467640988642,
 {'ada_tree__learning_rate': 1.1, 'ada_tree__n_estimators': 50})

# 6) Stack the best model from each model class

We manually input the best parameters from each gridsearched pipeline above. 

In [199]:
from sklearn.ensemble import StackingClassifier

In [200]:
level1_models = [
#     ('pipe_logreg', Pipeline([
#             ('ss',StandardScaler()),
#             ('logreg',LogisticRegression(penalty='l2' , max_iter=10**5) )])
#             ), #end pipe_logreg
    ('pipe_knn', Pipeline([
        ('ss',StandardScaler()),
        ('knn' , KNeighborsClassifier(n_neighbors=10,weights='uniform') )] )
            ), # end pipe_knn
    ('pipe_dtc' , Pipeline([
        ('ss',StandardScaler()),
        ('dtc',DecisionTreeClassifier(max_depth=5) )])
            ), # end pipe_dtc
    ('pipe_btc',Pipeline([
        ('ss',StandardScaler()),
        ('btc',BaggingClassifier(n_estimators=150) )
         ])
            ), #end bag
    ('pipe_rfc', Pipeline([
        ('ss',StandardScaler()),
        ('rfc',RandomForestClassifier(max_depth=10, n_estimators=200) )
         ])
            ), # end rfc 
#     ('pipe_ada_logreg' , Pipeline([
#         ('ss',StandardScaler()),
#         ('ada_logreg', AdaBoostClassifier(base_estimator=LogisticRegression(),
#                                             learning_rate=1.1, n_estimators = 10))
#          ])
#         ), #end adalog
    ('pipe_ada_tree' , Pipeline([
        ('ss',StandardScaler()),
        ('ada_tree', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),learning_rate =0.9,n_estimators=100) )
         ]) #end adatree
)
                ]
     



## 6) Fit Stack 

The following fits on augmented/balanced data, which does not provide the best results. 

In [201]:
balanced_stack = StackingClassifier(estimators=level1_models,
                         final_estimator=LogisticRegression())
balanced_stack.fit(X_train_bal,y_train_bal);

## 6.2) Fit on unbalanced data

In [202]:
unbalanced_stack = StackingClassifier(estimators=level1_models,
                         final_estimator=LogisticRegression())
unbalanced_stack.fit(X_train,y_train);

# 7) Predict 

Create the predictions to be submitted for the competition.

In [203]:
# reminder: because it was preprocessed with train and val data
# X_test = test_data.drop(columns=['wage']) above
preds = unbalanced_stack.predict(X_test)
probs = unbalanced_stack.predict_proba(X_test)

In [204]:
# checking index of test dataset was retained before creating a data frame 
test_data.index[:5], test_data.index[-5:]

(Int64Index([0, 1, 2, 3, 4], dtype='int64'),
 Int64Index([16276, 16277, 16278, 16279, 16280], dtype='int64'))

In [205]:
submission = pd.DataFrame(
                index = test_data.index, # index from test data
                columns=['preds','wage'] #required column names 
                         )
submission['preds'] = preds.astype(int)
submission['wage'] = probs

submission.head()

,preds,wage
0,0,0.968520
1,0,0.785319
2,0,0.764227
3,1,0.076408
4,0,0.968694


In [163]:
submission.to_csv( index=False)

# 8) Metrics

In [164]:
preds_val = unbalanced_stack.predict(X_val)
probs_val = unbalanced_stack.predict_proba(X_val)

## F1 score

This competition will be judge on F1 scores. Thus, we examine

In [165]:
from sklearn.metrics import f1_score

f1_score(y_val,preds_val) 

0.6468085106382978

## 7.4) Accuracy, Precision, Sensitivity, Specificity, 

In [166]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, balanced_accuracy_score # from which specificity can be computed
accuracy_score(y_val, preds_val)


0.8471454880294659

In [167]:
precision_score(y_val,preds_val)

0.7284345047923323

In [168]:
recall_score(y_val,preds_val) #aka sensitivity

0.5816326530612245

In [169]:
# specificity = 2*balanced - sensitivity
specificity = 2*balanced_accuracy_score(y_val,preds_val) - recall_score(y_val,preds_val) 
specificity

0.931285367825384

In [170]:
# if the binary classes had been labeled differently, the f1 would have been:
2*precision_score(y_val,preds_val)*specificity/(precision_score(y_val,preds_val)+specificity)

0.8174637261675787

## Train set, Val set Accuracy Score

In [171]:
%%time
unbalanced_stack.score(X_train,y_train)

CPU times: user 2.31 s, sys: 956 ms, total: 3.26 s
Wall time: 1.46 s


0.9882686084142395

In [172]:
unbalanced_stack.score(X_val,y_val)

0.8471454880294659

## Confusion Matrix 

In [ ]:
from sklearn.metrics import plot_confusion_matrix 

In [ ]:
plot_confusion_matrix(stack, 
                      X_val,y_val, 
                      cmap='ocean', 
                      display_labels=['<= $50k','> $50k'],
                      colorbar=False); 

## Area under ROC

In [ ]:
from sklearn import metrics
metrics.plot_roc_curve(stack, X_val,y_val)
plt.plot([0,1],[0,1], color='r', label='Baseline')
# add a legend
plt.legend();

In [ ]:

metrics.roc_auc_score(y_val,probs_val[:,1])